<a href="https://colab.research.google.com/github/Jepees/StepAhead-Nuevo/blob/main/scripts/1_Tokopedia_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kode ini saya buat sendiri, silahkan jika ingin digunakan atau dikembangkan, namun gunakan datanya hanya untuk tujuan penelitian

In [ ]:
import time
import pandas as pd
import json

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

# Functions

In [ ]:
def tokped_crawler(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    time.sleep(3)
    ActionChains(driver).move_by_offset(10, 10).click().perform()

    kategori = {'running': 'Lari',
                'lari': 'Lari',
                'futsal': 'futsal',
                'bola': 'bola',
                'football': 'bola',
                'soccer': 'bola',
                'sneakers': 'sneakers',
                'trail run': 'trail run',
                'basket': 'basket',
                }

    data_fix = {'nama barang': [],
            'kategori': [],
            'harga': [],
            'date': [],
            'user':[],
            'varian': [],
            'rating': [],
            'review': [],
            'bantu': [],
            }

    temp = None
    for i in range(100):
        soup = BeautifulSoup(driver.page_source, "html.parser")
        titles = soup.find('div', attrs={'class': 'css-j0ozid'})
        containers = soup.find_all('article', attrs={'class': 'css-15m2bcr'})

        if not containers:
            break

        data = {'nama barang': [],
            'kategori': [],
            'harga': [],
            'date': [],
            'user':[],
            'varian': [],
            'rating': [],
            'review': [],
            'bantu': [],
            }

        for container in containers:
            product = titles.find('p', attrs={'class': 'css-1oeizd2-unf-heading e1qvo2ff8'}).text
            catt = next((kategori[key] for key in kategori if key in product.lower()), None)
            price = titles.find('span', attrs={'class': 'css-rfqff4'}).text
            date = container.find('p', attrs={'class':'css-1rpz5os-unf-heading e1qvo2ff8'}).text
            user = container.find('span', attrs={'class':'name'}).text
            varian = None if container.find('p', attrs={'class':'css-5amcmn-unf-heading e1qvo2ff8'}) == None else container.find('p', attrs={'class':'css-5amcmn-unf-heading e1qvo2ff8'}).text[8:]
            rating = container.find('div', attrs={'data-testid':'icnStarRating'}).get('aria-label')
            review = None if container.find('span', attrs={'data-testid':'lblItemUlasan'}) == None else container.find('span', attrs={'data-testid':'lblItemUlasan'}).text.replace('\n', ' ')
            bantu = '0 orang terbantu' if container.find('span', attrs={'class':'css-q2y3yl'}).text == 'Membantu' else container.find('span', attrs={'class':'css-q2y3yl'}).text

            data['nama barang'].append(product)
            data['kategori'].append(catt)
            data['harga'].append(price)
            data['user'].append(user)
            data['date'].append(date)
            data['varian'].append(varian)
            data['rating'].append(rating)
            data['review'].append(review)
            data['bantu'].append(bantu)
            # print(f"{date}\n{user}\n{varian}\n{rating}\n{review}\n{bantu}\n")
        if temp == review:
            driver.quit()
            df = pd.DataFrame(data_fix)
            return df
        temp=review

        for key in data_fix.keys():
            data_fix[key] += data[key]
        print(f"total review {i+1} page: {len(data_fix['review'])}")

        time.sleep(1)
        try:
            driver.find_element(By.CSS_SELECTOR, "button[aria-label^='Laman berikutnya']").click()
        except:
            print('hanya ada 1 halaman')
            driver.quit()
            df = pd.DataFrame(data_fix)
            return df
        time.sleep(2)
    driver.quit()

    df = pd.DataFrame(data_fix)
    return df

def concater(urls):
    dfs = {}
    for i, url in enumerate(urls):
        print(f"produk ke-{i+1}: {url}" )
        dfs[f"df{i+1}"] = tokped_crawler(url)
        total = sum([i.shape[0] for i in dfs.values()])
        print(f"total seluruh review saat ini: {total}")
        print('')

    df = pd.concat(list(dfs.values()), ignore_index=True)
    print(f"total review {df.shape[0]}")
    return df

# Crawl Nuevo

In [ ]:
# code untuk menangkap semua link review produk di etalase store tokped

base_url = 'https://www.tokopedia.com/nuevo-offcl-shop'
urls_nuevo = []

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
driver.get(base_url)

halaman = 1
while True:
    print(f"Halaman {halaman}")
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    soup = BeautifulSoup(driver.page_source, "html.parser")
    containers = soup.find_all('div', attrs={'class': 'css-79elbk'})
    for container in containers:
        x = container.find('a', attrs={'class': 'oQ94Awb6LlTiGByQZo8Lyw== IM26HEnTb-krJayD-R0OHw=='})
        if x:
            prod = x.get('href')
            review_link = f"{prod.split('?')[0]}/review"
            urls_nuevo.append(review_link)
    print(f"total produk saat ini: {len(urls_nuevo)}")

    next_link_tag = soup.find('a', attrs={'data-testid': 'btnShopProductPageNext'})
    if next_link_tag:
        next_href = next_link_tag.get('href')
        full_url = "https://www.tokopedia.com" + next_href
        print(f"Pindah ke: {full_url}\n")
        driver.get(full_url)
        time.sleep(1)
        halaman += 1
    else:
        print("Tidak ada halaman berikutnya. Selesai.\n")
        break

driver.quit()

# Hapus duplikat & tampilkan hasil
urls_nuevo = sorted(set(urls_nuevo))
print(f"Total link review dikumpulkan: {len(urls_nuevo)}")

with open('urls_nuevo.json', 'w', encoding='utf-8') as f:
    json.dump(urls_nuevo, f, ensure_ascii=False, indent=2)

Halaman 1
total produk saat ini: 18
Tidak ada halaman berikutnya. Selesai.

Total link review dikumpulkan: 18


In [ ]:
df_nuevo = concater(urls_nuevo)
df_nuevo.to_csv("Tokopedia_review_Nuevo.csv", index=False)
df_nuevo

produk ke-1: https://www.tokopedia.com/nuevo-offcl-shop/nuevo-crospeed-navy-stabilo-sepatu-running-olahraga-lari-outdoor-1729582107005323019/review
total review 1 page: 10
total review 2 page: 20
total review 3 page: 30
total review 4 page: 40
total review 5 page: 50
total review 6 page: 60
total review 7 page: 63
total seluruh review saat ini: 63

produk ke-2: https://www.tokopedia.com/nuevo-offcl-shop/nuevo-crosspeed-hitam-purpls-sepatu-trail-running-beginner-sneakers-olahraga-lari-pria-outdoor-1729582046604003083/review
total review 1 page: 10
total review 2 page: 20
total review 3 page: 30
total review 4 page: 40
total review 5 page: 50
total review 6 page: 60
total review 7 page: 70
total review 8 page: 80
total review 9 page: 90
total review 10 page: 97
total seluruh review saat ini: 160

produk ke-3: https://www.tokopedia.com/nuevo-offcl-shop/nuevo-runastic-putih-hitam-sepatu-sneakers-running-sport-shoes-olahraga-lari-pria-outdoor-1729566242372946699/review
total review 1 page: 

,nama barang,kategori,harga,date,user,varian,rating,review,bantu
0,Nuevo Crospeed Navy/Stabilo Sepatu Running Ola...,Lari,Rp153.600,8 bulan lalu,A***s,Navy Ijo - 41,bintang 4,Bentuk dan ukuran sepatu sesuai. Kualitas luma...,0 orang terbantu
1,Nuevo Crospeed Navy/Stabilo Sepatu Running Ola...,Lari,Rp153.600,8 bulan lalu,K***i,Navy Ijo - 42,bintang 5,"Sejauh ini bagus, tinggal review pemakaian aja...",0 orang terbantu
2,Nuevo Crospeed Navy/Stabilo Sepatu Running Ola...,Lari,Rp153.600,3 bulan lalu,J***w,Navy Ijo - 41,bintang 4,Seoatunya bagus banget ga rugi beli nyaman ban...,0 orang terbantu
3,Nuevo Crospeed Navy/Stabilo Sepatu Running Ola...,Lari,Rp153.600,8 bulan lalu,_***,Navy Ijo - 42,bintang 4,Bgus lah Ada Harga Ad Kualitas saran Sya brng ...,0 orang terbantu
4,Nuevo Crospeed Navy/Stabilo Sepatu Running Ola...,Lari,Rp153.600,10 bulan lalu,r***9,Abu Oren - 40,bintang 5,Nyaman dipakainya dan kualitasnya bagus. Terim...,0 orang terbantu
...,...,...,...,...,...,...,...,...,...
2341,Sepatu Sneakers Pria Wanita Siloka Shadow Low ...,sneakers,Rp59.900,Lebih dari 1 tahun lalu,A***e,38,bintang 4,None,0 orang terbantu
2342,Sepatu Sneakers Pria Wanita Siloka Shadow Low ...,sneakers,Rp59.900,Lebih dari 1 tahun lalu,D***i,40,bintang 5,None,0 orang terbantu
2343,Sepatu Sneakers Pria Wanita Siloka Shadow Low ...,sneakers,Rp59.900,Lebih dari 1 tahun lalu,d***a,42,bintang 5,Lumyan,0 orang terbantu
2344,Sepatu Sneakers Pria Wanita Siloka Shadow Low ...,sneakers,Rp59.900,Lebih dari 1 tahun lalu,r***n,41,bintang 3,None,0 orang terbantu


# Crawl Mills

In [ ]:
base_url = 'https://www.tokopedia.com/mills-official-978/product?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=3875195&srp_page_title=MILLS%20OFFICIAL&navsource=shop'
urls_mills = []

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
driver.get(base_url)

halaman = 1
while True:
    print(f"Halaman {halaman}")
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    soup = BeautifulSoup(driver.page_source, "html.parser")
    containers = soup.find_all('div', attrs={'class': 'css-79elbk'})
    for container in containers:
        x = container.find('a', attrs={'class': 'oQ94Awb6LlTiGByQZo8Lyw== IM26HEnTb-krJayD-R0OHw=='})
        if x:
            prod = x.get('href')
            review_link = f"{prod.split('?')[0]}/review"
            urls_mills.append(review_link)
    print(f"total produk saat ini: {len(urls_mills)}")

    next_link_tag = soup.find('a', attrs={'data-testid': 'btnShopProductPageNext'})
    if next_link_tag:
        next_href = next_link_tag.get('href')
        full_url = "https://www.tokopedia.com" + next_href
        print(f"Pindah ke: {full_url}\n")
        driver.get(full_url)
        time.sleep(1)
        halaman += 1
    else:
        print("Tidak ada halaman berikutnya. Selesai.\n")
        break

driver.quit()

# Hapus duplikat & tampilkan hasil
urls_mills = sorted(set(urls_mills))
print(f"Total link review dikumpulkan: {len(urls_mills)}")

with open('urls_mills.json', 'w', encoding='utf-8') as f:
    json.dump(urls_mills, f, ensure_ascii=False, indent=2)

Halaman 1
total produk saat ini: 80
Pindah ke: https://www.tokopedia.com/mills-official-978/product/page/2?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=3875195&srp_page_title=MILLS%20OFFICIAL&navsource=shop

Halaman 2
total produk saat ini: 160
Pindah ke: https://www.tokopedia.com/mills-official-978/product/page/3?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=3875195&srp_page_title=MILLS%20OFFICIAL&navsource=shop

Halaman 3
total produk saat ini: 240
Pindah ke: https://www.tokopedia.com/mills-official-978/product/page/4?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=3875195&srp_page_title=MILLS%20OFFICIAL&navsource=shop

Halaman 4
total produk saat ini: 314
Tidak ada halaman berikutnya. Selesai.

Total link review dikumpulkan: 314


In [ ]:
df_mills = concater(urls_mills)
df_mills.to_csv("Tokopedia_review_Mills.csv", index=False)
df_mills

produk ke-1: https://www.tokopedia.com/mills-official-978/mills-sepatu-dexter-black-black-9701802-1729589330515625761/review
total review 1 page: 10
total review 2 page: 20
total review 3 page: 30
total review 4 page: 33
total seluruh review saat ini: 33

produk ke-2: https://www.tokopedia.com/mills-official-978/mills-sepatu-dexter-black-white-9701801-1729589386518693665/review
total review 1 page: 7
hanya ada 1 halaman
total seluruh review saat ini: 40

produk ke-3: https://www.tokopedia.com/mills-official-978/mills-sepatu-evander-black-dark-grey-red-9700803-1729650443580443425/review
total review 1 page: 10
total review 2 page: 20
total review 3 page: 30
total review 4 page: 39
total seluruh review saat ini: 79

produk ke-4: https://www.tokopedia.com/mills-official-978/mills-sepatu-evander-black-white-9700802-black-white-36-5ef01/review
total review 1 page: 10
total review 2 page: 20
total seluruh review saat ini: 99

produk ke-5: https://www.tokopedia.com/mills-official-978/mills-se

,nama barang,kategori,harga,date,user,varian,rating,review,bantu
0,MILLS Sepatu Dexter Black / Black 9701802 - BL...,None,Rp329.000,3 minggu lalu,Muhammad,BLACK/BLACK - 42,bintang 5,"Ok sih bagus, cuma kurang suka sama bekas puti...",0 orang terbantu
1,MILLS Sepatu Dexter Black / Black 9701802 - BL...,None,Rp329.000,1 bulan lalu,N***t,BLACK/BLACK - 41,bintang 5,Keren sepatu empuk vuat kaki lebar cocok yaa,0 orang terbantu
2,MILLS Sepatu Dexter Black / Black 9701802 - BL...,None,Rp329.000,2 bulan lalu,S***o,BLACK/BLACK - 37,bintang 5,"Mantap, enak dipake dan pengiriman kilat.. Bi...",0 orang terbantu
3,MILLS Sepatu Dexter Black / Black 9701802 - BL...,None,Rp329.000,1 minggu lalu,H***l,BLACK/BLACK - 42,bintang 5,"barang: ori kualitas: bahan bagus, ringan, ti...",0 orang terbantu
4,MILLS Sepatu Dexter Black / Black 9701802 - BL...,None,Rp329.000,2 bulan lalu,Z***k,BLACK/BLACK - 42,bintang 5,Bentuk dan ukuran: Pas Aroma: Sodappp Daya tah...,0 orang terbantu
...,...,...,...,...,...,...,...,...,...
2689,Mills Sepatu Lari Running Shoes Enermax Dynapl...,Lari,Rp549.000,7 bulan lalu,G***a,RBlue/Org/Wht - 37,bintang 5,None,1 orang terbantu
2690,Mills Sepatu Lari Running Shoes Enermax Dynapl...,Lari,Rp549.000,6 bulan lalu,Dyana,RBlue/Org/Wht - 40,bintang 5,mantap. enggak yg empuk2 bgt tapi so far so go...,0 orang terbantu
2691,Mills Sepatu Lari Running Shoes Enermax Dynapl...,Lari,Rp549.000,1 bulan lalu,M***d,RBlue/Org/Wht - 40,bintang 5,Toppppppppppppppp ! Sukses berkah selaluu gan!,0 orang terbantu
2692,Mills Sepatu Lari Running Shoes Enermax Dynapl...,Lari,Rp549.000,1 bulan lalu,Q***p,RBlue/Org/Wht - 45,bintang 5,So far so good,0 orang terbantu


# Crawl Ortuseight

In [ ]:
base_url = 'https://www.tokopedia.com/ortuseightofficial/product?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=10999364&srp_page_title=Ortuseight&navsource=shop'
urls_ortuseight = []

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
driver.get(base_url)

halaman = 1
while True:
    print(f"Halaman {halaman}")
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    soup = BeautifulSoup(driver.page_source, "html.parser")
    containers = soup.find_all('div', attrs={'class': 'css-79elbk'})
    for container in containers:
        x = container.find('a', attrs={'class': 'oQ94Awb6LlTiGByQZo8Lyw== IM26HEnTb-krJayD-R0OHw=='})
        if x:
            prod = x.get('href')
            review_link = f"{prod.split('?')[0]}/review"
            urls_ortuseight.append(review_link)
    print(f"total produk saat ini: {len(urls_ortuseight)}")

    next_link_tag = soup.find('a', attrs={'data-testid': 'btnShopProductPageNext'})
    if next_link_tag:
        next_href = next_link_tag.get('href')
        full_url = "https://www.tokopedia.com" + next_href
        print(f"Pindah ke: {full_url}\n")
        driver.get(full_url)
        time.sleep(1)
        halaman += 1
    else:
        print("Tidak ada halaman berikutnya. Selesai.\n")
        break

driver.quit()

# Hapus duplikat & tampilkan hasil
urls_ortuseight = sorted(set(urls_ortuseight))
print(f"Total link review dikumpulkan: {len(urls_ortuseight)}")

with open('urls_ortuseight.json', 'w', encoding='utf-8') as f:
    json.dump(urls_ortuseight, f, ensure_ascii=False, indent=2)

Halaman 1
total produk saat ini: 80
Pindah ke: https://www.tokopedia.com/ortuseightofficial/product/page/2?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=10999364&srp_page_title=Ortuseight&navsource=shop

Halaman 2
total produk saat ini: 160
Pindah ke: https://www.tokopedia.com/ortuseightofficial/product/page/3?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=10999364&srp_page_title=Ortuseight&navsource=shop

Halaman 3
total produk saat ini: 240
Pindah ke: https://www.tokopedia.com/ortuseightofficial/product/page/4?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=10999364&srp_page_title=Ortuseight&navsource=shop

Halaman 4
total produk saat ini: 247
Tidak ada halaman berikutnya. Selesai.

Total link review dikumpulkan: 247


In [ ]:
df_ortuseight = concater(urls_ortuseight)
df_ortuseight.to_csv("Tokopedia_review_Ortuseight.csv", index=False)
df_ortuseight

produk ke-1: https://www.tokopedia.com/ortuseightofficial/ortuseight-sepatu-catalyst-empire-in-distilled-blue-camo-1730693476600743686/review
total review 1 page: 9
hanya ada 1 halaman
total seluruh review saat ini: 9

produk ke-2: https://www.tokopedia.com/ortuseightofficial/ortuseight-sepatu-futsal-catalyst-ballistic-in-cloud-blue-cyan-1730534015650662150/review
total review 1 page: 10
total review 2 page: 20
total review 3 page: 23
total seluruh review saat ini: 32

produk ke-3: https://www.tokopedia.com/ortuseightofficial/ortuseight-sepatu-futsal-catalyst-ballistic-in-mint-grey-1731586872481646342/review
total review 1 page: 1
hanya ada 1 halaman
total seluruh review saat ini: 33

produk ke-4: https://www.tokopedia.com/ortuseightofficial/ortuseight-sepatu-futsal-catalyst-ballistic-in-silver-black-1730833586551817990/review
total review 1 page: 10
total seluruh review saat ini: 43

produk ke-5: https://www.tokopedia.com/ortuseightofficial/ortuseight-sepatu-futsal-catalyst-ballistic-

,nama barang,kategori,harga,date,user,varian,rating,review,bantu
0,Ortuseight Sepatu Catalyst Empire IN Distilled...,None,Rp699.000,5 bulan lalu,h***0,Distilled Blue Camo - 42,bintang 3,Paket udah sampai sangat cepat. Agak kecewa aj...,0 orang terbantu
1,Ortuseight Sepatu Catalyst Empire IN Distilled...,None,Rp699.000,6 bulan lalu,Asep,Distilled Blue - 43,bintang 5,Nyaman dan ringan,0 orang terbantu
2,Ortuseight Sepatu Catalyst Empire IN Distilled...,None,Rp699.000,4 bulan lalu,A***f,Distilled Blue Camo - 44,bintang 5,Bentuk dan ukuran: Passs MANTAPP,0 orang terbantu
3,Ortuseight Sepatu Catalyst Empire IN Distilled...,None,Rp699.000,6 bulan lalu,*****,Distilled Blue Camo - 37,bintang 5,Top banget ringan,1 orang terbantu
4,Ortuseight Sepatu Catalyst Empire IN Distilled...,None,Rp699.000,6 bulan lalu,ð****,Distilled Blue Camo - 39,bintang 5,kece badaii🔥,0 orang terbantu
...,...,...,...,...,...,...,...,...,...
5736,Sepatu Sepak Bola Catalyst Liberte V4 FG Salmo...,bola,Rp649.000,2 minggu lalu,K***n,Salmon Lime - 41,bintang 5,Mantap,0 orang terbantu
5737,Sepatu Sepak Bola Catalyst Liberte V4 FG Salmo...,bola,Rp649.000,2 minggu lalu,S***a,Salmon Lime - 41,bintang 4,Good,0 orang terbantu
5738,Sepatu Sepak Bola Catalyst Liberte V4 FG Salmo...,bola,Rp649.000,2 minggu lalu,G***t,Salmon Lime - 39,bintang 5,Nice,0 orang terbantu
5739,Sepatu Sepak Bola Catalyst Liberte V4 FG Salmo...,bola,Rp649.000,1 minggu lalu,X***b,Salmon Lime - 41,bintang 5,Keren,0 orang terbantu


# Crawl 910

In [ ]:
base_url = 'https://www.tokopedia.com/910ninetenshoes/product?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=1114391&srp_page_title=910%20NINETEN%20SHOES&navsource=shop'
urls_910 = []

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
driver.get(base_url)

halaman = 1
while True:
    print(f"Halaman {halaman}")
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    soup = BeautifulSoup(driver.page_source, "html.parser")
    containers = soup.find_all('div', attrs={'class': 'css-79elbk'})
    for container in containers:
        x = container.find('a', attrs={'class': 'oQ94Awb6LlTiGByQZo8Lyw== IM26HEnTb-krJayD-R0OHw=='})
        if x:
            prod = x.get('href')
            review_link = f"{prod.split('?')[0]}/review"
            urls_910.append(review_link)
    print(f"total produk saat ini: {len(urls_910)}")

    next_link_tag = soup.find('a', attrs={'data-testid': 'btnShopProductPageNext'})
    if next_link_tag:
        next_href = next_link_tag.get('href')
        full_url = "https://www.tokopedia.com" + next_href
        print(f"Pindah ke: {full_url}\n")
        driver.get(full_url)
        time.sleep(1)
        halaman += 1
    else:
        print("Tidak ada halaman berikutnya. Selesai.\n")
        break

driver.quit()

# Hapus duplikat & tampilkan hasil
urls_910 = sorted(set(urls_910))
print(f"Total link review dikumpulkan: {len(urls_910)}")

with open('urls_910.json', 'w', encoding='utf-8') as f:
    json.dump(urls_910, f, ensure_ascii=False, indent=2)

Halaman 1
total produk saat ini: 80
Pindah ke: https://www.tokopedia.com/910ninetenshoes/product/page/2?q=sepatu&srp_component_id=02.01.00.00&srp_page_id=1114391&srp_page_title=910%20NINETEN%20SHOES&navsource=shop

Halaman 2
total produk saat ini: 114
Tidak ada halaman berikutnya. Selesai.

Total link review dikumpulkan: 114


In [ ]:
df_910 = concater(urls_910)
df_910.to_csv("Tokopedia_review_910.csv", index=False)
df_910

produk ke-1: https://www.tokopedia.com/910ninetenshoes/910-haze-strike-prime-sepatu-lari-putih-biru-mint-1731748869083202775/review
total review 1 page: 8
hanya ada 1 halaman
total seluruh review saat ini: 8

produk ke-2: https://www.tokopedia.com/910ninetenshoes/910-nineten-aurorun-sepatu-lari-hitam-biru-ungu-olahraga-running-outdoor-1729620355022161111/review
total review 1 page: 10
total review 2 page: 20
total review 3 page: 30
total review 4 page: 40
total review 5 page: 50
total review 6 page: 58
total seluruh review saat ini: 66

produk ke-3: https://www.tokopedia.com/910ninetenshoes/910-nineten-aurorun-stride-sepatu-lari-hijau-mint-hijau-tosca-putiih-gading-1731811001560237271/review
total seluruh review saat ini: 66

produk ke-4: https://www.tokopedia.com/910ninetenshoes/910-nineten-aurorun-stride-sepatu-lari-hitam-abu-abu-tua-1731809955341370583/review
total review 1 page: 4
hanya ada 1 halaman
total seluruh review saat ini: 70

produk ke-5: https://www.tokopedia.com/910ninet

,nama barang,kategori,harga,date,user,varian,rating,review,bantu
0,910 Haze STRIKE PRIME Sepatu Lari - Putih/Biru...,Lari,Rp550.905,2 hari lalu,A***s,42,bintang 5,Sepatunya ok dan pas dengan ukuran.. Nyampenya...,0 orang terbantu
1,910 Haze STRIKE PRIME Sepatu Lari - Putih/Biru...,Lari,Rp550.905,1 hari lalu,Z***i,42,bintang 5,barang original enak di pakek ngebut sepatunya,0 orang terbantu
2,910 Haze STRIKE PRIME Sepatu Lari - Putih/Biru...,Lari,Rp550.905,1 minggu lalu,R***i,45,bintang 5,Bagus nyaman melebihi ekspektasi saya,0 orang terbantu
3,910 Haze STRIKE PRIME Sepatu Lari - Putih/Biru...,Lari,Rp550.905,2 minggu lalu,V***u,41,bintang 5,Produknya mantapppplll,1 orang terbantu
4,910 Haze STRIKE PRIME Sepatu Lari - Putih/Biru...,Lari,Rp550.905,3 hari lalu,Y***e,40,bintang 5,Pengiriman cepat,0 orang terbantu
...,...,...,...,...,...,...,...,...,...
3136,910Nineten TAKASHI RUN 1.5 Sepatu Lari - Kunin...,Lari,Rp645.905,7 bulan lalu,Dalia,43,bintang 5,"warna bagus,dipakai lari enak",0 orang terbantu
3137,910Nineten TAKASHI RUN 1.5 Sepatu Lari - Kunin...,Lari,Rp645.905,2 bulan lalu,b***5,44,bintang 5,Pengiriman cepat,0 orang terbantu
3138,910Nineten TAKASHI RUN 1.5 Sepatu Lari - Kunin...,Lari,Rp645.905,5 bulan lalu,T***S,44,bintang 5,Mantap,0 orang terbantu
3139,910Nineten TAKASHI RUN 1.5 Sepatu Lari - Kunin...,Lari,Rp645.905,6 bulan lalu,â****,40,bintang 5,Mantap,0 orang terbantu
